In [73]:
import pandas as pd
import datetime

from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize

def to_date(s):
    dt = datetime.datetime.fromtimestamp(s / 1e3)
    return dt.strftime("%Y-%m-%d")

begin_time='2019-01-07'
end_time = '2019-05-27'
cal_period = '7D'


dsl = {
        'query': {'bool': {'must': [{'range': {'paytime': {'format': 'yyyy-MM-dd',
                                                           'gte': begin_time,
                                                           'lte': end_time}}},
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    {'terms':{'user_member_type': ['black_card_vip']}},
                                    {'terms':{'platform_count': ['1']}}
                                    ]}},
        
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": '1w', 'time_zone': '+00:00'}}},
                        {"用户ID": {"terms": {"field": 'user_id'}}},
                        {"会员": {"terms": {"field": 'user_member_type'}}},
                        {"平台消费次数": {"terms": {"field": 'platform_count'}}},
                    ]
                },
                'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}}}
                
            }
        }
        
    }
es = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res = es.search(index='es_order_table', doc_type='order', body=dsl,
                    _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                    search_type='query_then_fetch')
df = json_normalize(res['aggregations']['my_buckets']['buckets'])
df.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df.columns))
df['日期'] = df['日期'].apply(to_date)
df['日期'] = pd.to_datetime(df['日期'])

#df_new.to_csv('abc.csv')
df.head()

,doc_count,会员,平台消费次数,日期,用户ID,折前GMV.value
0,1,black_card_vip,1,2019-01-07,168155,0.01
1,1,black_card_vip,1,2019-01-07,176007,0.01
2,1,black_card_vip,1,2019-01-07,176008,59.00
3,1,black_card_vip,1,2019-01-07,176012,0.10
4,1,black_card_vip,1,2019-01-07,181397,10.00


In [74]:
def week_new_user_list(s):
    data = {}
    data['new_user_list'] = s['用户ID'].unique().tolist()
    return  pd.Series(data)

df =  df.resample('7d', on = '日期').apply(week_new_user_list).reset_index()
df

,日期,new_user_list
0,2019-01-07,"[168155, 176007, 176008, 176012, 181397, 18584..."
1,2019-01-14,"[158750, 176362, 188640, 194703, 197883, 22013..."
2,2019-01-21,"[177321, 188522, 192045, 197847, 232480, 23284..."
3,2019-01-28,"[199918, 241504, 242902, 245218, 245742, 24784..."
4,2019-02-04,"[184725, 255247, 256366, 270315, 304501, 30578..."
5,2019-02-11,"[177911, 236578, 238024, 240865, 245937, 25098..."
6,2019-02-18,"[180530, 196579, 233131, 241393, 241646, 24205..."
7,2019-02-25,"[185813, 189046, 192939, 251658, 271370, 29660..."
8,2019-03-04,"[165943, 176327, 242612, 242615, 246173, 24619..."
9,2019-03-11,"[190322, 237650, 240087, 242850, 251740, 25314..."


In [75]:
this_user_count_list,this_gmv_count_list,this_order_count_list=[],[],[]
for i in range(len(df)):
    begin_time = str(df.iloc[i]['日期'])[:10]
    new_user_list = df.iloc[i]['new_user_list']
    end_time = str(datetime.datetime.strptime(begin_time,'%Y-%m-%d') + datetime.timedelta(days=6))[:10]
    dsl_new = {
        'query': {'bool': {'must': [{'range': {'paytime': {'format': 'yyyy-MM-dd',
                                                           'gte': begin_time,
                                                           'lte': end_time}}},
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    {'terms':{'user_member_type': ['black_card_vip']}},
                                    {'terms':{'user_id': new_user_list}}
                                    ]}},
        
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": '1w', 'time_zone': '+00:00'}}},
                        
                        #{"会员": {"terms": {"field": 'user_member_type'}}},
                        #{"平台消费次数": {"terms": {"field": 'platform_count'}}},
                    ]
                },
                'aggregations': {
                  #  '会员数': {'cardinality': {'field': 'user_id'}},
                    '折前GMV': {'sum': {'field': 'skutotal_price'}},
                    '订单数':{'cardinality':{'field':'order_id'}}
                    
              }      
            }
        }
        
    }
    es_new = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_new = es_new.search(index='es_order_table', doc_type='order', body=dsl_new,
                    _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                    search_type='query_then_fetch')
    df_new = json_normalize(res_new['aggregations']['my_buckets']['buckets'])
    if len(res_new['aggregations']['my_buckets']['buckets'])>0:
        #user_count = res_new['aggregations']['my_buckets']['buckets'][0]['会员数']['value']
        user_count = len(new_user_list)
        gmv_count = res_new['aggregations']['my_buckets']['buckets'][0]['折前GMV']['value']
        order_count = res_new['aggregations']['my_buckets']['buckets'][0]['doc_count']
    else:
        user_count = gmv_count = order_count =0
        
    this_user_count_list.append(user_count)
    this_gmv_count_list.append(gmv_count)
    this_order_count_list.append(order_count)
    
df['本周会员数'] = this_user_count_list
df['本周GMV'] = this_gmv_count_list
df['本周订单数'] = this_order_count_list

df

,日期,new_user_list,本周会员数,本周GMV,本周订单数
0,2019-01-07,"[168155, 176007, 176008, 176012, 181397, 18584...",40,1293.760001,123
1,2019-01-14,"[158750, 176362, 188640, 194703, 197883, 22013...",42,2414.710000,65
2,2019-01-21,"[177321, 188522, 192045, 197847, 232480, 23284...",928,24148.020002,1266
3,2019-01-28,"[199918, 241504, 242902, 245218, 245742, 24784...",494,17984.799985,683
4,2019-02-04,"[184725, 255247, 256366, 270315, 304501, 30578...",23,706.500000,25
5,2019-02-11,"[177911, 236578, 238024, 240865, 245937, 25098...",137,8638.600000,226
6,2019-02-18,"[180530, 196579, 233131, 241393, 241646, 24205...",106,5396.100000,167
7,2019-02-25,"[185813, 189046, 192939, 251658, 271370, 29660...",185,14384.300001,318
8,2019-03-04,"[165943, 176327, 242612, 242615, 246173, 24619...",247,23224.610000,380
9,2019-03-11,"[190322, 237650, 240087, 242850, 251740, 25314...",236,22404.090000,326


In [101]:
user_count_list,gmv_count_list,order_count_list=[],[],[]
for i in range(len(df)):
    date = str(df.iloc[i]['日期'])[:10]
    user_list = df.iloc[i]['new_user_list']
    interval = 9
    begin_date = str(datetime.datetime.strptime(date,'%Y-%m-%d')+datetime.timedelta(days=7*interval))[:10]
    end_date = str(datetime.datetime.strptime(date,'%Y-%m-%d')+datetime.timedelta(days=7*interval+6))[:10]
    dsl_after = {
        'query': {'bool': {'must': [{'range': {'paytime': {'format': 'yyyy-MM-dd',
                                                           'gte': begin_date,
                                                           'lte': end_date}}},
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    {'terms':{'user_member_type': ['black_card_vip']}},
                                    {'terms':{'user_id': user_list}}
                                    ]}},
        
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": '1w', 'time_zone': '+00:00'}}},
                    ]
                },
                'aggregations': {
                    '会员数': {'cardinality': {'field': 'user_id'}},
                    '折前GMV': {'sum': {'field': 'skutotal_price'}},
                    '订单数':{'cardinality':{'field':'order_id'}}
              }      
            }
        }
        
    }
    es_after = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_after = es_after.search(index='es_order_table', doc_type='order', body=dsl_after,
                _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                search_type='query_then_fetch')
    
    if len(res_after['aggregations']['my_buckets']['buckets'])>0:
        user_count = res_after['aggregations']['my_buckets']['buckets'][0]['会员数']['value']
        #user_count = 
        gmv_count = res_after['aggregations']['my_buckets']['buckets'][0]['折前GMV']['value']
        order_count = res_after['aggregations']['my_buckets']['buckets'][0]['订单数']['value']
    else:
        user_count = gmv_count = order_count =0
    user_count_list.append(user_count)
    gmv_count_list.append(gmv_count)
    order_count_list.append(order_count)
df[str(interval)+'周后会员数'] = user_count_list
df[str(interval)+'周后GMV'] = gmv_count_list
df[str(interval)+'周后订单数'] = order_count_list


df

,日期,new_user_list,本周会员数,本周GMV,本周订单数,1周后会员数,1周后GMV,1周后订单数,2周后会员数,2周后GMV,...,5周后订单数,6周后会员数,6周后GMV,6周后订单数,7周后会员数,7周后GMV,7周后订单数,9周后会员数,9周后GMV,9周后订单数
0,2019-01-07,"[168155, 176007, 176008, 176012, 181397, 18584...",40,1293.760001,123,18,1134.190000,59,23,1372.270000,...,13,9,196.340000,11,9,562.500000,15,10,364.100000,16
1,2019-01-14,"[158750, 176362, 188640, 194703, 197883, 22013...",42,2414.710000,65,22,1626.070000,50,18,1766.500000,...,14,10,480.500000,18,9,419.000000,10,11,442.000000,15
2,2019-01-21,"[177321, 188522, 192045, 197847, 232480, 23284...",928,24148.020002,1266,156,7858.970000,276,22,1353.500000,...,183,92,5862.000000,146,86,5443.900000,126,82,5459.859997,154
3,2019-01-28,"[199918, 241504, 242902, 245218, 245742, 24784...",494,17984.799985,683,21,1071.000000,24,63,4529.599999,...,82,53,4234.300001,102,47,3685.300000,75,53,3269.980000,84
4,2019-02-04,"[184725, 255247, 256366, 270315, 304501, 30578...",23,706.500000,25,5,188.000000,8,3,105.000000,...,10,4,122.000000,5,1,82.000000,1,3,354.000000,4
5,2019-02-11,"[177911, 236578, 238024, 240865, 245937, 25098...",137,8638.600000,226,36,1951.849999,59,32,2392.550000,...,41,27,1892.250001,76,24,1640.530000,58,36,4058.809996,88
6,2019-02-18,"[180530, 196579, 233131, 241393, 241646, 24205...",106,5396.100000,167,23,1226.800000,44,22,1371.000000,...,35,11,428.000000,12,16,725.399999,27,33,1825.599993,62
7,2019-02-25,"[185813, 189046, 192939, 251658, 271370, 29660...",185,14384.300001,318,51,4803.799999,93,36,3047.000000,...,58,29,1680.779999,61,54,3841.499994,110,56,4954.519994,109
8,2019-03-04,"[165943, 176327, 242612, 242615, 246173, 24619...",247,23224.610000,380,55,4510.600000,94,39,2825.800000,...,56,62,3709.799991,106,77,6648.809996,171,73,5192.700000,155
9,2019-03-11,"[190322, 237650, 240087, 242850, 251740, 25314...",236,22404.090000,326,52,4393.910001,86,51,2329.700000,...,95,76,5794.650000,140,67,3761.059992,120,71,5270.199994,131


In [102]:
df.to_excel(str(interval)+'周会员留存率.xlsx')

In [58]:
dsl_new = {
        'query': {'bool': {'must': [{'range': {'paytime': {'format': 'yyyy-MM-dd',
                                                           'gte': begin_time,
                                                           'lte': end_time}}},
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    {'terms':{'user_member_type': ['black_card_vip']}},
                                    {'terms':{'user_id': df['用户ID'].tolist()}}
                                    ]}},
        
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": '1w', 'time_zone': '+00:00'}}},
                        {"用户ID": {"terms": {"field": 'user_id'}}}
                        #{"会员": {"terms": {"field": 'user_member_type'}}},
                        #{"平台消费次数": {"terms": {"field": 'platform_count'}}},
                    ]
                },
                'aggregations': {
                    '折前GMV': {'sum': {'field': 'skutotal_price'}},
                    #'订单数':{'cardinality':{'field':'order_id'}}
              }      
            }
        }
        
    }
es_new = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res_new = es_new.search(index='es_order_table', doc_type='order', body=dsl_new,
                    _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                    search_type='query_then_fetch')
df_new = json_normalize(res_new['aggregations']['my_buckets']['buckets'])
df_new.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_new.columns))
df_new['日期'] = df_new['日期'].apply(to_date)
df_new['日期'] = pd.to_datetime(df_new['日期'])
df_new

KeyError: '用户ID'

In [123]:
#df_new.to_csv('zxc.csv')     #最开始的结束

In [130]:
import pandas as pd
import datetime

from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize

def to_date(s):
    dt = datetime.datetime.fromtimestamp(s / 1e3)
    return dt.strftime("%Y-%m-%d")





begin_time='2019-01-07'
end_time = '2019-05-27'
cal_period = '7D'

dsl_later = {
        'query': {'bool': {'must': [{'range': {'paytime': {'format': 'yyyy-MM-dd',
                                                           'gte': begin_time,
                                                           'lte': end_time}}},
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    {'terms':{'user_member_type': ['black_card_vip']}},
                                    ]}},
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": "1d"}}},
                        {"用户ID": {"terms": {"field": 'user_id'}}},
                        {"会员": {"terms": {"field": 'user_member_type'}}},
                        {"平台消费次数": {"terms": {"field": 'platform_count'}}},
                    ]
                }
            }
        }
    }
es_later = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res_later = es_later.search(index='es_order_table', doc_type='order', body=dsl_later,
                    _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                    search_type='query_then_fetch')
df_later = json_normalize(res_later['aggregations']['my_buckets']['buckets'])
df_later.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_later.columns))
df_later['日期'] = df_later['日期'].apply(to_date)
df_later['日期'] = pd.to_datetime(df_later['日期'])

In [131]:
only_new_vip = True
cal_period = '7D'

In [132]:
def get_num(s):
        new_restriction = df_later['平台消费次数'] == 1
        vip_restriction = df_later['会员'] == 'black_card_vip'
        if only_new_vip:
            data = {'the_user_list': s['用户ID'].tolist(), 'the_new_user_list': s[new_restriction&vip_restriction]['用户ID'].tolist()}
        elif only_vip:
            data = {'the_user_list': s['用户ID'].tolist(), 'the_new_user_list': s[vip_restriction]['用户ID'].tolist()}
        else:
            data = {'the_user_list': s['用户ID'].tolist(), 'the_new_user_list': s['用户ID'].tolist()}
        return pd.Series(data)
    
    

dts_later = df_later.resample(cal_period, on='日期').apply(get_num).reset_index()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [154]:
cal_interval=8
dts_later['the_new_user_list_upper'] = dts_later['the_new_user_list'].shift(cal_interval)
dts_later = dts_later.fillna(0)

In [155]:
new_list=[]
for i, row in dts_later.iterrows():
    if row.the_new_user_list_upper != 0:
        new_list.append(set(row.the_user_list)&set(row.the_new_user_list_upper))


In [156]:
qwe=[]
for i,item in enumerate(new_list, start=0):
    dsl_it = {
        'query': {'bool': {'must': [{'range': {'paytime': {'format': 'yyyy-MM-dd',
                                                           'gte': begin_time,
                                                           'lte': end_time}}},
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    {'terms':{'user_member_type': ['black_card_vip']}},
                                    {'terms':{'user_id': list(item)}}
                                    ]}},
        
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": '1w', 'time_zone': '+00:00'}}},
                        #{"用户ID": {"terms": {"field": 'user_id'}}},
                        #{"会员": {"terms": {"field": 'user_member_type'}}},
                        #{"平台消费次数": {"terms": {"field": 'platform_count'}}},
                    ]
                },
                'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}}}
                
            }
        }
        
    }
    es_it = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res_it = es_it.search(index='es_order_table', doc_type='order', body=dsl_it,
                    _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                    search_type='query_then_fetch')
    df_it = json_normalize(res_it['aggregations']['my_buckets']['buckets'])
    df_it.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df_it.columns))
    df_it['日期'] = df_it['日期'].apply(to_date)
    df_it['日期'] = pd.to_datetime(df_it['日期'])
    qwe.append({i,df_it.iloc[0]['doc_count']/df_new.iloc[i]['doc_count'],  df_it.iloc[0]['折前GMV.value']/df_new.iloc[i]['折前GMV.value']})
    print(i,df_it.iloc[0]['doc_count']/df_new.iloc[i]['doc_count'],  df_it.iloc[0]['折前GMV.value']/df_new.iloc[i]['折前GMV.value'])

0 0.13008130081300814 0.14585394498572432
1 0.11290322580645161 0.14694694130581842
2 0.09705882352941177 0.10316189720326027
3 0.0909952606635071 0.13278753087349066
4 0.03296703296703297 0.016052237790776764
5 0.06427221172022685 0.06008139253893757
6 0.122568093385214 0.10201879947005517
7 0.12613784135240572 0.12980465259654736
8 0.1420256111757858 0.13338887176258374
9 0.14183551847437426 0.13235006425971252
10 0.15949632738719832 0.12476023858979873
11 0.13808574277168495 0.1105628735082731
12 0.022587268993839837 0.014358646865074846
